In [1]:
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Set ERCOT load zone and solar capacity
ERCOT_LOAD_ZONE = 'LZ_HOUSTON'
SOLAR_CAPACITY = 200
SOLAR_PPA_DOL_KWH = 250
FACILITY_LIFETIME = 25


(CVXPY) Apr 14 03:21:56 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Apr 14 03:21:56 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


/Users/jacklee-kopij/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/jacklee-kopij/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/var/folders/y8/n0cwg6hn0xv21zz309nl1hv40000gn/T/ipykernel_14904/1267442481.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
 

In [2]:
# READ IN DATA
# TODO: update as 'electricity' here is a normalised 8760 solar profile
hourly_dayahead_realtime_solar = pd.read_csv("hourly_dayahead_realtime_solar.csv")

# Filter by load zone and complete data
ERCOT_LOAD_ZONE = 'LZ_HOUSTON'
hourly_dayahead_realtime_solar_lz = hourly_dayahead_realtime_solar[hourly_dayahead_realtime_solar['zone'] == ERCOT_LOAD_ZONE]
hourly_dayahead_realtime_solar_lz = hourly_dayahead_realtime_solar_lz[hourly_dayahead_realtime_solar_lz['electricity'].notna()]

hourly_dayahead_realtime_solar_lz = hourly_dayahead_realtime_solar_lz.iloc[0:24,]
# Sets time duration of the optimisation
# TODO: generat appropriate 8760 - due to shift in UTC time used for solar profile 
# and local_time used for ERCOT realtime and dayahead prices
T = hourly_dayahead_realtime_solar_lz.shape[0]

In [3]:
# FACILITY
FACILITY_LIFETIME = 25
OFFTAKE_TPD = 10

# POWER
#   SOLAR
SOLAR_CAPACITY = 200
SOLAR_PPA_DOL_KWH = 0.22 # 50 cents per kWh ($USD 50 per MWh)
hourly_solar_production_kwh = hourly_dayahead_realtime_solar_lz['electricity']
#   GRID 
#       CONVERT $/MWh -> $/kWh
hourly_dayahead_dol_per_kwh = hourly_dayahead_realtime_solar_lz['price_dayahead'] / 1000


# EQUIPMENT
#   OPERATIONAL 
#       EFFICIENCY
ELECTROLYSIS_EFFICIENCY = 55
COMPRESSION_EFFICIENCY = 4
LIQUEFACTION_EFFICIENCY = 10
#       LOWER OPERATION BOUNDS (% of nameplate capacity)
ELECTROLYSIS_OPERATION_LOWER = 0.15
COMPRESSOR_OPERATION_LOWER = 0.15
LIQUEFACTION_OPERATION_LOWER = 0.60 
#       RAMP CONSTRAINTS (% of nameplate capacity)
LIQUEFACTION_MAX_UP = 0.1
LIQUEFACTION_MAX_DOWN = 0.1

#   CAPEX
#   CAPEX - total
ELECTROLYSIS_CAPEX_DOL_TPD = 1200000 # 1.2 million $USD per TPD of electrolysis
ELECTROLYSIS_CAPEX_DOL_TPD = 800000 # 0.8 million $USD per TPD of comrpessor
LIQUEFACTION_CAPEX_DOL_TPD = 2350000 # 2.35 million $USD per TPD of liquefaction per day
#   CAPEX - annualised
ELECTROLYSIS_ANNUALISED_CAPEX_DOL_TPD = ELECTROLYSIS_CAPEX_DOL_TPD / FACILITY_LIFETIME
COMPRESSOR_ANNUALISED_CAPEX_DOL_TPD = ELECTROLYSIS_CAPEX_DOL_TPD / FACILITY_LIFETIME
LIQUEFACTION_ANNUALISED_CAPEX_DOL_TPD = LIQUEFACTION_CAPEX_DOL_TPD / FACILITY_LIFETIME
#   CAPEX - daily, for fast testing 
ELECTROLYSIS_DAY_CAPEX_DOL_TPD = ELECTROLYSIS_CAPEX_DOL_TPD / (FACILITY_LIFETIME * 365)
COMPRESSOR_DAY_CAPEX_DOL_TPD = ELECTROLYSIS_CAPEX_DOL_TPD / (FACILITY_LIFETIME * 365)
LIQUEFACTION_DAY_CAPEX_DOL_TPD = LIQUEFACTION_CAPEX_DOL_TPD / (FACILITY_LIFETIME * 365)


In [4]:
# add effinciencies 
# ramping constraints 

import cvxpy as cp




# Define decision variables
#   Electricity decision variables
realtime_consumption = cp.Variable(T, nonneg = True)
realtime_supplied = cp.Variable(T, nonneg = True)
solar_consumed_facility = cp.Variable(T, nonneg = True) 

#   Design decision variables
electrolyser_nameplate_capacity_hour = cp.Variable(nonneg = True)
compressor_nameplate_capacity_hour = cp.Variable(nonneg = True)
liquefaction_nameplate_capacity_hour = cp.Variable(nonneg = True)

# set throughput
electrolyser_throughput = cp.Variable(T, nonneg = True)
compressor_throughput = cp.Variable(T, nonneg = True)
compressor_to_liquefaction = cp.Variable(T, nonneg = True)
liquefacion_throughput = cp.Variable(T, nonneg = True)

#   Operational decision variables
electrolyser_throttle = cp.Variable(T)
compression_throttle = cp.Variable(T)
liquefaction_throttle = cp.Variable(T)

gh2_storage_inflow = cp.Variable(T,nonneg=True)
gh2_storage_outflow = cp.Variable(T,nonneg=True)
gh2_storage_level = cp.Variable(T+1, nonneg = True)
lh2_storage_level = cp.Variable(T+1, nonneg = True)
gh2_storage_active = cp.Variable(T, boolean=True)

# Define power balance constraints t
constraints = [
    # restrict the amount of solar that can be consumed
    solar_consumed_facility <= SOLAR_CAPACITY *
    cp.sum(realtime_consumption) + cp.sum(solar_consumed_facility) <= 10000, 
    # power balance equation: sum of electricity in = amount consumed           
    realtime_consumption + solar_consumed_facility == 
        ELECTROLYSIS_EFFICIENCY*electrolyser_throughput + 
        COMPRESSION_EFFICIENCY*compressor_throughput + 
        LIQUEFACTION_EFFICIENCY*liquefacion_throughput, 
]
# operational constraints of equipment
constraints += [
    # constraint operational bounds of each piece of equipment: 
    0 <= electrolyser_throughput, 
    electrolyser_throughput <= electrolyser_nameplate_capacity_hour,
    0 <= compressor_throughput, 
    compressor_throughput <= compressor_nameplate_capacity_hour,
    0 <= liquefacion_throughput, 
    liquefacion_throughput <= liquefaction_nameplate_capacity_hour
    ]

# ramp constraints throughput constraints 
constraints += [ 
    liquefacion_throughput[1:T] - liquefacion_throughput[0:T-1] <= LIQUEFACTION_MAX_UP*liquefaction_nameplate_capacity_hour,
    liquefacion_throughput[0:T-1] - liquefacion_throughput[1:T] <= LIQUEFACTION_MAX_DOWN*liquefaction_nameplate_capacity_hour,
]

# set initial storage levels to empty

"""***test that starting with non-zero storage will turn off eletrolysis and compression"""
# constraints += [ 
#     gh2_storage_level[10] == 1
# ]
constraints += [ 
    gh2_storage_level[0] == 0,
    # gh2_storage_level[10] == 1, # test to 
]


# balance mass through system
constraints += [
    electrolyser_throughput == compressor_throughput,
    compressor_throughput == compressor_to_liquefaction + gh2_storage_inflow,
    liquefacion_throughput == compressor_to_liquefaction + gh2_storage_outflow,
    # liquefacion_throughput == compressor_throughput + gh2_storage_netflow,
    gh2_storage_level[1:T+1] == gh2_storage_level[0:T] + gh2_storage_inflow  - gh2_storage_outflow,
    # big-M constraint to only turn on storage if 
    compressor_throughput - liquefacion_throughput <= 1000 * gh2_storage_active,
    gh2_storage_inflow <= 1000*gh2_storage_active,
    gh2_storage_outflow <= 1000*gh2_storage_active
]

# set 1 tph offtake 
constraints += [ 
    sum(liquefacion_throughput) == cp.Parameter(T, value=[OFFTAKE_TPD]*T)
]

# Minimize total cost of supply; min(OPEX + CAPEX)
# OPEX = electricity, CAPEX = annualised capex
objective = 0

# add electricity cost from dayahead market consumption
objective += realtime_consumption @ hourly_dayahead_dol_per_kwh * 100000 

# add electricity cost from solar ppa consumption
objective += cp.sum(SOLAR_PPA_DOL_KWH * solar_consumed_facility)

# add capex costs for electrolyser, compressor and liquefaction
objective += electrolyser_nameplate_capacity_hour * ELECTROLYSIS_DAY_CAPEX_DOL_TPD
objective += compressor_nameplate_capacity_hour * COMPRESSOR_DAY_CAPEX_DOL_TPD
objective += liquefaction_nameplate_capacity_hour * LIQUEFACTION_DAY_CAPEX_DOL_TPD


# Define problem
objective_min = cp.Minimize(objective)
problem = cp.Problem(objective_min, constraints)

# Solve problem
problem.solve()


# Print solution
# print("Optimal value:", problem.value)
# print("Optimal variable:", realtime_consumption.value)
# print(f"solar consumed {solar_consumed_facility.value}")
# print(f"Max throughput: {electrolyser_nameplate_capacity_hour.value}")
# print("elec")
# print(electrolyser_throughput.value)
# print("compressor")
power_opex_total = np.sum(hourly_dayahead_dol_per_kwh @ realtime_consumption.value)
power_opex_per_kg = power_opex_total/sum(liquefacion_throughput.value)

capex_total = electrolyser_nameplate_capacity_hour.value * ELECTROLYSIS_DAY_CAPEX_DOL_TPD
capex_per_kg = capex_total / sum(liquefacion_throughput.value)
print(f"power opex / kg {power_opex_per_kg}")
print(f"capex per kg {capex_per_kg}")
print(f"total dol per kg {power_opex_per_kg + capex_per_kg}")

Exception: Cannot evaluate the truth value of a constraint or chain constraints, e.g., 1 >= x >= 0.

In [ ]:

# create dataframe of results: 
operations_df = pd.DataFrame({
    "realtime_price_$_per_kwh": hourly_dayahead_dol_per_kwh,
    "realtime_production_kwh":realtime_consumption.value,
    "solar_production_kwh": solar_consumed_facility.value, 
    "electrolyser_consumption_kwh": ELECTROLYSIS_EFFICIENCY*electrolyser_throughput.value, 
    "compressor_consumption_kwh": COMPRESSION_EFFICIENCY*compressor_throughput.value, 
    "liquefaction_consumption_kwh": LIQUEFACTION_EFFICIENCY*liquefacion_throughput.value,
    "electrolyser_produced_kg": electrolyser_throughput.value, 
    "compressor_produced_kg": compressor_throughput.value, 
    "liquefaction_produced_kg": liquefacion_throughput.value,
    "compress_to_liquefaction": compressor_to_liquefaction.value,
    "gh2_storage_level_kg": gh2_storage_level.value[0:T],
    # TODO: if problem size allows, add in big-M constraint 
    "gh2_storge_inflow_kg": gh2_storage_inflow.value[0:T], 
    "gh2_storage_outflow_kg": gh2_storage_outflow.value[0:T]
    # "gh2_storage_netflow": gh2_storage_netflow.value
})
operations_df.to_csv("operations_8760_df.csv")

In [ ]:
hourly_dayahead_dol_per_kwh